# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.5446, -0.5908],
         [ 1.1910,  0.0506],
         [ 1.5655, -0.5934],
         [-1.4086,  0.5528],
         [ 0.4821, -0.4015],
         [-0.5308,  0.6325],
         [-0.8063, -2.3190],
         [-0.4961, -0.4519],
         [ 1.8696, -0.5931],
         [ 0.5584, -1.3583]]),
 tensor([[ 7.3077],
         [ 6.4149],
         [ 9.3590],
         [-0.5005],
         [ 6.5200],
         [ 0.9647],
         [10.4692],
         [ 4.7394],
         [ 9.9590],
         [ 9.9396]])]

使用框架的预定义好的层

In [10]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [20]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [16]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [19]:
trainer = torch.optim.SGD(net.parameters(), lr=0.01)

训练过程代码与我们从零开始实现时所做的非常相似

In [21]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.564221
epoch 2, loss 0.009627
epoch 3, loss 0.000246
epoch 4, loss 0.000096
epoch 5, loss 0.000093
epoch 6, loss 0.000093
epoch 7, loss 0.000093
epoch 8, loss 0.000093
epoch 9, loss 0.000093
epoch 10, loss 0.000093


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [22]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003, -0.0002])
b的估计误差： tensor([9.5367e-05])
